<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=00059fda34339f53398329ccf2f695fc898e634c35ebb1e45338238a3c67f29e
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.0
    Uninstalling protobuf-6.32.0:
      Successfully uninstalled protobuf-6.32.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-09-05 10:19:52
-------------------
qualified stocks: 89
with latest results: 26
still star stocks: 15
-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  7.27 L
Current:  1.39 C
-------------------
Today PnL: -46.18 K (-0.33%)
Current PnL: -18.57 L (-12.49%)
CY Booked + Current PnL: -8.72 L (-5.86%)
-------------------
Total profit:  2.86 L
Total loss:  -21.44 L
-------------------
Total Booked + Current PnL: 18.26 L (15.01%)
Total Booked PnL: 36.84 L (30.27%)
Curr Year Booked PnL: 9.86 L (7.07%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.19 C
Est FTT PnL: 79.29 L (56.85%)
Deployed:  1.22 C
Current:  1.39 C
CAGR/XIRR %: 8.6%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,DIXON,18940.15,-28.58,71.0,H-MC,7.43,142888.0,27952.0,8630.0,0.05,24.32,6.04,31.83,81.0,3.24,1.08,48.11,X40N,ATH,IT
4,APOLLOHOSP,8285.00,-13.20,63.0,H-LC,10.10,173514.0,17337.0,8762.0,-0.16,11.10,5.05,16.71,22.0,1.98,1.31,30.47,X40N,BTT,HEALTHCARE
79,TTKPRESTIG,770.00,106.19,67.0,M-SC,6.35,88844.0,-11933.0,12029.0,0.41,-11.84,13.54,0.09,245.0,-0.99,0.67,16.72,OX40N,NTT,DURABLES
64,SBIN,863.00,-13.25,45.0,M-LC,4.49,213616.0,12136.0,15081.0,-0.41,6.02,7.06,13.51,60.0,0.80,1.62,19.38,XY25,NTT,BANKS
26,DMART,5201.00,-5.31,72.0,H-LC,15.54,236489.0,52008.0,18352.0,-0.06,28.19,7.76,38.14,38.0,2.83,1.79,41.79,X40N,NTT,FMCG


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
37,ICICIPRULI,790.00,-21.34,37.0,H-MC,2.20,135419.0,-369.0,43117.0,-0.10,-0.27,31.84,31.48,107.0,-0.01,1.02,11.81,X40,ATH,INSURANCE
36,ICICIGI,2260.25,-20.34,42.0,H-MC,4.15,137152.0,1019.0,30105.0,-0.97,0.75,21.95,22.86,91.0,0.03,1.04,13.46,X40,ATH,INSURANCE
53,PGHH,17973.08,-30.02,59.0,H-MC,4.98,202725.0,1905.0,66879.0,-0.07,0.95,32.99,34.25,80.0,0.03,1.53,6.20,X40,ATH,FMCG
88,WIPRO,420.00,-14.70,39.0,M-LC,6.98,150189.0,-756.0,110209.0,-1.11,-0.50,73.38,72.51,53.0,-0.01,1.14,5.49,XR,NTT,IT


In [11]:
# top 5 to enter based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,HCLTECH,1943.91,-3.23,31.0,H-LC,10.01,221197.0,-20727.0,83989.0,-2.38,-8.57,37.97,26.15,8.0,-0.25,1.67,4.53,X40,ATH,IT
76,TCS,4998.00,-28.36,39.0,H-LC,14.41,285196.0,-60760.0,184607.0,-1.99,-17.56,64.73,35.80,1.0,-0.33,2.16,1.03,X40,BTT,IT
85,VBL,671.64,-14.77,40.0,H-LC,6.90,307165.0,-8677.0,121330.0,-1.64,-2.75,39.50,35.67,5.0,-0.07,2.32,10.73,X40N,ATH,FMCG
45,JIOFIN,387.00,-1.56,41.0,H-LC,13.99,217700.0,294.0,53206.0,0.50,0.14,24.44,24.61,48.0,0.01,1.65,55.03,XY24,BTT,FINANCE
58,RELIANCE,1533.00,-14.79,42.0,H-LC,7.23,212940.0,2574.0,26213.0,0.42,1.22,12.31,13.68,37.0,0.10,1.61,17.95,XY25,NTT,REFINERIES


In [12]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
49,LTIM,7230.20,-7.80,44.0,H-LC,2.01,228578.0,-17573.0,96780.0,-2.50,-7.14,42.34,32.18,16.0,-0.18,1.73,26.33,X200,ATH,IT
52,NESTLEIND,1377.00,-8.73,60.0,H-LC,2.54,281011.0,15585.0,41196.0,-0.96,5.87,14.66,21.40,11.0,0.38,2.13,12.90,XY25,NTT,FMCG
43,ITC,452.00,-38.34,47.0,H-LC,3.06,198876.0,-1262.0,22155.0,-2.21,-0.63,11.14,10.44,4.0,-0.06,1.50,4.90,X40,NTT,FMCG
19,CIPLA,1795.00,-19.09,57.0,H-LC,5.61,216049.0,11549.0,29858.0,-0.05,5.65,13.82,20.25,10.0,0.39,1.63,14.87,X40N,BTT,PHARMA
85,VBL,671.64,-14.77,40.0,H-LC,6.90,307165.0,-8677.0,121330.0,-1.64,-2.75,39.50,35.67,5.0,-0.07,2.32,10.73,X40N,ATH,FMCG


In [13]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
7,ATULAUTO,844.00,4008.11,79.0,M-SC,9.59,128018.0,-19350.0,71165.0,6.55,-13.13,55.59,35.16,236.0,-0.27,0.97,30.76,XY24,NTT,AUTO
87,WHIRLPOOL,2270.00,-39.15,63.0,M-SC,4.22,102840.0,11342.0,67412.0,1.62,12.40,65.55,86.07,223.0,0.17,0.78,49.34,XR,NTT,DURABLES
69,SONACOMS,1006.00,-37.48,46.0,M-SC,12.72,77761.0,-23398.0,98290.0,1.46,-23.13,126.40,74.03,202.0,-0.24,0.59,9.65,AR,BTT,AUTO
86,VOLTAS,1918.49,1.58,63.0,H-MC,4.01,213120.0,21378.0,74656.0,1.30,11.15,35.03,50.08,99.0,0.29,1.61,18.70,XY25,ATH,AC
84,VALIANTORG,838.00,-308.98,43.0,H-SC,9.56,88920.0,-44366.0,128961.0,1.18,-33.29,145.03,63.47,139.0,-0.34,0.67,49.05,XR,NTT,CHEMICALS


In [14]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
49,LTIM,7230.20,-7.80,44.0,H-LC,2.01,228578.0,-17573.0,96780.0,-2.50,-7.14,42.34,32.18,16.0,-0.18,1.73,26.33,X200,ATH,IT
32,HCLTECH,1943.91,-3.23,31.0,H-LC,10.01,221197.0,-20727.0,83989.0,-2.38,-8.57,37.97,26.15,8.0,-0.25,1.67,4.53,X40,ATH,IT
68,SIS,528.00,2184.46,44.0,H-SC,5.69,61455.0,-19635.0,28306.0,-2.26,-24.21,46.06,10.69,156.0,-0.69,0.46,24.55,OX40N,NTT,MISC
43,ITC,452.00,-38.34,47.0,H-LC,3.06,198876.0,-1262.0,22155.0,-2.21,-0.63,11.14,10.44,4.0,-0.06,1.50,4.90,X40,NTT,FMCG
76,TCS,4998.00,-28.36,39.0,H-LC,14.41,285196.0,-60760.0,184607.0,-1.99,-17.56,64.73,35.80,1.0,-0.33,2.16,1.03,X40,BTT,IT


In [15]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
79,TTKPRESTIG,770.0,106.19,67.0,M-SC,6.35,88844.0,-11933.0,12029.0,0.41,-11.84,13.54,0.09,245.0,-0.99,0.67,16.72,OX40N,NTT,DURABLES
80,UJJIVANSFB,60.0,83.20,43.0,H-SC,13.19,115695.0,-26784.0,46301.0,-0.60,-18.80,40.02,13.70,163.0,-0.58,0.88,36.55,OX40N,NTT,BANKS
46,KANSAINER,340.0,-68.34,54.0,H-SC,3.48,218655.0,-51012.0,87353.0,-0.12,-18.92,39.95,13.47,138.0,-0.58,1.65,10.59,XY24,NTT,PAINTS
39,INDIGOPNTS,1408.0,115.40,51.0,M-SC,3.54,141248.0,-33311.0,33349.0,-1.19,-19.08,23.61,0.02,221.0,-1.00,1.07,22.86,OX40N,NTT,PAINTS
18,CERA,9475.0,-19.81,45.0,H-SC,2.77,114246.0,-29557.0,56300.0,-0.28,-20.55,49.28,18.60,149.0,-0.52,0.86,26.07,OX40N,NTT,CERAMICS


In [16]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TCS,4998.00,-28.36,39.0,H-LC,14.41,285196.0,-60760.0,184607.0,-1.99,-17.56,64.73,35.80,1.0,-0.33,2.16,1.03,X40,BTT,IT
41,INFY,2275.00,-19.96,43.0,H-LC,11.10,311619.0,-1269.0,172949.0,-1.09,-0.41,55.50,54.87,3.0,-0.01,2.36,6.22,X40,BTT,IT
43,ITC,452.00,-38.34,47.0,H-LC,3.06,198876.0,-1262.0,22155.0,-2.21,-0.63,11.14,10.44,4.0,-0.06,1.50,4.90,X40,NTT,FMCG
85,VBL,671.64,-14.77,40.0,H-LC,6.90,307165.0,-8677.0,121330.0,-1.64,-2.75,39.50,35.67,5.0,-0.07,2.32,10.73,X40N,ATH,FMCG
1,ABB,7934.00,-41.33,43.0,H-LC,13.13,245011.0,-16608.0,135810.0,-1.13,-6.35,55.43,45.57,7.0,-0.12,1.85,4.25,AR,NTT,ELECTRICAL


In [17]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
33,HEROMOTOCO,5949.07,1.28,78.0,H-MC,21.10,188055.0,37142.0,20159.0,0.41,24.61,10.72,37.97,93.0,1.84,1.42,56.14,AR,ATH,AUTO
25,DIXON,18940.15,-28.58,71.0,H-MC,7.43,142888.0,27952.0,8630.0,0.05,24.32,6.04,31.83,81.0,3.24,1.08,48.11,X40N,ATH,IT
26,DMART,5201.00,-5.31,72.0,H-LC,15.54,236489.0,52008.0,18352.0,-0.06,28.19,7.76,38.14,38.0,2.83,1.79,41.79,X40N,NTT,FMCG


In [18]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
14,BLUESTARCO,2326.38,11.77,60.0,H-MC,3.35,192940.0,28270.0,39707.0,-0.75,17.17,20.58,41.28,89.0,0.71,1.46,26.52,X40N,ATH,AC
86,VOLTAS,1918.49,1.58,63.0,H-MC,4.01,213120.0,21378.0,74656.0,1.30,11.15,35.03,50.08,99.0,0.29,1.61,18.70,XY25,ATH,AC
33,HEROMOTOCO,5949.07,1.28,78.0,H-MC,21.10,188055.0,37142.0,20159.0,0.41,24.61,10.72,37.97,93.0,1.84,1.42,56.14,AR,ATH,AUTO
23,DABUR,735.00,-1.31,66.0,H-MC,4.46,214773.0,17745.0,71885.0,-0.36,9.01,33.47,45.49,102.0,0.25,1.62,23.22,XY24,BTT,FMCG
17,CAMS,5211.76,-6.49,47.0,H-SC,5.52,106330.0,4326.0,39597.0,0.87,4.24,37.24,43.06,122.0,0.11,0.80,22.08,X40N,ATH,MISC


In [19]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
33,HEROMOTOCO,5949.07,1.28,78.0,H-MC,21.10,188055.0,37142.0,20159.0,0.41,24.61,10.72,37.97,93.0,1.84,1.42,56.14,AR,ATH,AUTO
51,NATIONALUM,247.44,-45.32,71.0,H-MC,0.55,109842.0,9338.0,21298.0,0.28,9.29,19.39,30.49,79.0,0.44,0.83,44.71,MH,ATH,METALS
87,WHIRLPOOL,2270.00,-39.15,63.0,M-SC,4.22,102840.0,11342.0,67412.0,1.62,12.40,65.55,86.07,223.0,0.17,0.78,49.34,XR,NTT,DURABLES
38,INDIAMART,4850.92,-51.84,44.0,H-SC,12.21,134673.0,11337.0,122431.0,-0.79,9.19,90.91,108.45,119.0,0.09,1.02,33.60,AR,ATH,MISC
88,WIPRO,420.00,-14.70,39.0,M-LC,6.98,150189.0,-756.0,110209.0,-1.11,-0.50,73.38,72.51,53.0,-0.01,1.14,5.49,XR,NTT,IT


In [20]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
3,ALKYLAMINE,4546.37,-4.17,45.0,H-SC,15.65,89199.0,-11764.0,106298.0,-1.18,-11.65,119.17,93.63,148.0,-0.11,0.67,32.86,SR,ATH,CHEMICALS
21,COFFEEDAY,80.00,-39.64,61.0,L-SC,36.90,88454.0,-25095.0,65146.0,-0.32,-22.10,73.65,35.27,268.0,-0.39,0.67,116.49,XR,NTT,HOTELS
50,MASFIN,398.61,-18.70,48.0,H-SC,12.86,92415.0,-5565.0,27170.0,-0.84,-5.68,29.40,22.05,152.0,-0.20,0.70,34.33,XR,ATH,FINANCE
51,NATIONALUM,247.44,-45.32,71.0,H-MC,0.55,109842.0,9338.0,21298.0,0.28,9.29,19.39,30.49,79.0,0.44,0.83,44.71,MH,ATH,METALS
80,UJJIVANSFB,60.00,83.20,43.0,H-SC,13.19,115695.0,-26784.0,46301.0,-0.60,-18.80,40.02,13.70,163.0,-0.58,0.88,36.55,OX40N,NTT,BANKS


In [21]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,COFFEEDAY,80.00,-39.64,61.0,L-SC,36.90,88454.0,-25095.0,65146.0,-0.32,-22.10,73.65,35.27,268.0,-0.39,0.67,116.49,XR,NTT,HOTELS
7,ATULAUTO,844.00,4008.11,79.0,M-SC,9.59,128018.0,-19350.0,71165.0,6.55,-13.13,55.59,35.16,236.0,-0.27,0.97,30.76,XY24,NTT,AUTO
33,HEROMOTOCO,5949.07,1.28,78.0,H-MC,21.10,188055.0,37142.0,20159.0,0.41,24.61,10.72,37.97,93.0,1.84,1.42,56.14,AR,ATH,AUTO
57,RELAXO,1176.00,-31.65,61.0,H-SC,4.98,63443.0,-52095.0,87082.0,-0.53,-45.09,137.26,30.28,136.0,-0.60,0.48,24.73,X40N,NTT,FOOTWEAR
26,DMART,5201.00,-5.31,72.0,H-LC,15.54,236489.0,52008.0,18352.0,-0.06,28.19,7.76,38.14,38.0,2.83,1.79,41.79,X40N,NTT,FMCG


In [22]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.80
1,25,43.74
2,50,73.86


In [23]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    37.92
LC    36.22
MC    25.83
Name: CurrAlloc%, dtype: float64

In [24]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     27.78
X40      15.63
X40N     14.41
XY25     10.12
AR        9.51
XR        9.06
OX40N     7.44
MH        1.75
X200      1.73
X5K       1.44
SR        1.10
Name: CurrAlloc%, dtype: float64

In [25]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-LC    27.95
H-MC    22.84
H-SC    22.58
M-SC    13.75
M-LC     7.18
M-MC     2.67
L-SC     1.59
L-LC     1.09
L-MC     0.32
Name: CurrAlloc%, dtype: float64

In [26]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,13.97,0.21,34.04
IT,13.38,-16.77,74.67
FINANCE,7.42,-22.97,68.31
BANKS,7.39,-16.85,66.46
PAINTS,6.09,-14.68,40.35
MISC,5.85,-18.56,86.53
ELECTRICAL,5.71,-8.91,68.93
AUTO,4.87,-5.06,50.77
AC,3.80,3.73,28.87


In [27]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2793052.0,22
AR,1221040.0,10
XR,1096332.0,13
X40,899415.0,10
X40N,526609.0,11
OX40N,473766.0,10
XY25,413838.0,7
SR,197029.0,2
X5K,127389.0,1


In [28]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2786602.0,27
M-SC,1428039.0,17
H-LC,1424940.0,16
H-MC,1181571.0,17
M-LC,431241.0,5
M-MC,324925.0,2
L-SC,251500.0,3
L-MC,59735.0,1
L-LC,40202.0,1


In [29]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,,sum,count
Conviction,Criteria,,
H-SC,XY24,935102.0,6
M-SC,XY24,843867.0,7
H-SC,AR,663921.0,5
H-LC,X40,610703.0,5
H-SC,XR,527648.0,6
H-MC,XY24,457390.0,4
H-LC,AR,376751.0,2
M-MC,XY24,324925.0,2
H-LC,X40N,220091.0,5


In [30]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 56.0 seconds
